<a href="https://colab.research.google.com/github/antocommi/provagcp/blob/master/ProgettoAIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install -q mmcv
# !rm -rf mmaction2
# !git clone https://github.com/open-mmlab/mmaction2.git
# %cd mmaction2 
# !pip install -r -q requirements/build.txt
# !pip install -q -v -e .  # or "python setup.py develop"
# %cd ..

# # save state of colab
# #from google.colab import drive
# #drive.mount('/content/gdrive')
# #!pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt
#ciao prova
# import state of colab
from google.colab import drive
drive.mount('/content/gdrive')
!pip install -q --upgrade --force-reinstall `cat /content/gdrive/My\ Drive/colab_installed.txt`

     |████████████████████████████████| 235kB 8.9MB/s 
     |████████████████████████████████| 194kB 28.2MB/s 
Cloning into 'mmaction2'...
remote: Enumerating objects: 9261, done.
remote: Total 9261 (delta 0), reused 0 (delta 0), pack-reused 9261
Receiving objects: 100% (9261/9261), 35.28 MiB | 27.33 MiB/s, done.
Resolving deltas: 100% (6572/6572), done.
/content/mmaction2
ERROR: Invalid requirement: 'requirements/build.txt'
Hint: It looks like a path. It does exist. The argument you provided (requirements/build.txt) appears to be a requirements file. If that is the case, use the '-r' flag to install the packages specified within it.
Obtaining file:///content/mmaction2
  Running setup.py develop for mmaction2
/content
Mounted at /content/gdrive
     |████████████████████████████████| 133kB 8.6MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 737kB 12.0MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |██████████████████

In [ ]:
SEED = 9126

import os, cv2, json, time, math, sys, pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

from PIL import Image
import pandas as pd
import numpy as np
np.random.seed(SEED)
import seaborn as sn

# Check MMAction2 installation
import mmaction

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold

Importing from Kaggle
====

In [ ]:
# Download and load in memory dataset from kaggle
!mkdir ~/.kaggle #create the .kaggle folder in your root directory
!echo '{"username":"antocommii","key":"87dcebd7c4cb4ba4539ed72f027fcbde"}' > ~/.kaggle/kaggle.json #write kaggle API credentials to kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  # set permissions
!pip install -q kaggle #install the kaggle library
!kaggle datasets download --unzip --force antocommii/spacejam-action-recognition -p /content/kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 624M/626M [00:05<00:00, 163MB/s]
100% 626M/626M [00:05<00:00, 124MB/s]


Scelgo nuova size e definisco la struttura del Custom Dataset
====

In [ ]:
# Definisco nuove dimensioni dei frame dei video per velocizzare la rete
scale_factor_h, scale_factor_w = 0.637, 0.875 # 0.32, 0.44 -> 56,56 || 0.637, 0.875 -> 112,112
new_H = math.floor(176 * scale_factor_h) # 176
new_W = math.floor(128 * scale_factor_w) # 128
print("New Size: ",new_H,new_W)

New Size:  112 112


In [ ]:
new_H = new_W = 224

class CustomSet(torch.utils.data.Dataset):
    def __init__(self, videos_dir, list_dataset):
        """ Construct an indexed list of video paths and labels """
        
        self.VIDEO_DIR = videos_dir
        
        self.labels_dict = {0 : "block", 1 : "pass", 2 : "run", 3: "dribble",4: "shoot",
          5 : "ball in hand", 6 : "defense", 7: "pick" , 8 : "no_action" , 
          9: "walk" ,10: "discard"}

        self.dataset = list_dataset

    def __getitem__(self, index, is_for_testing=True):
        """ Load video n in the list of image paths and return it along with its label.
            In the case of multiclass the label will probably be a list of values"""
        
        name, label = self.dataset[index]
        
        fname = os.path.join(self.VIDEO_DIR, name+'.mp4')
        transform = transforms.Compose([
                                        transforms.Resize((new_H,new_W)),
                                        transforms.ToTensor()])
        video = []
        print("frames sampling enabled")
        accept = True
        vid = cv2.VideoCapture(fname)
        while True:
            # Capture frame-by-frame
            ret, frame = vid.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = transform(Image.fromarray(frame))
                frame = frame.unsqueeze(0)
                if accept:
                  video.append(frame)
                  accept=False
                else:
                  accept=True
            else:
              #print("------------Non trovato!---------------")
              break

        video = torch.cat(video, axis=0)
        video = video.permute([1,0,2,3])
        return {'video':video, 'label':torch.LongTensor([label])}
        

    def __len__(self):
        """ return the total number of video in this dataset """
        return len(self.dataset)

    def get_list(self):
      return self.dataset

Creazione Train, Validation e Test set
====

In [ ]:
# abbiamo 10 classi
num_classes = 10

# il dataset è ciò che si ottiene dal metodo precedente
ROOT_DIR = "/content/kaggle/"
VIDEO_DIR = "/content/kaggle/examples/"
ANNOTATION_FILE = "annotation_dict.json"
TEST_SET_FILE = "testset_keys_1lug2020.txt"

# prendo il dataset da annotation file
with open(os.path.join(ROOT_DIR, ANNOTATION_FILE)) as fp:
  annotations = json.load(fp)
            
with open(os.path.join(ROOT_DIR, TEST_SET_FILE)) as fp:
  keys_test = json.load(fp)

# divido in train e test second quello che ho nel file
annotationTrain = dict(filter(lambda x: x[0] not in keys_test, annotations.items()))
annotationTest = dict(filter(lambda x: x[0] in keys_test, annotations.items()))
test_set = list(annotationTest.items())

# divido in train e validation
train_set, valid_set = train_test_split(list(annotationTrain.items()), test_size=0.1)

#########################################################################
#########################################################################
# SOLO PER TEST #########################################################
#train_set = train_set[:2000]  ###########################################
#valid_set = valid_set[:200]   ###########################################
#test_set = test_set[:100]     ###########################################
#########################################################################
#########################################################################

# trasformo in custom_dataset
train_ds = CustomSet(VIDEO_DIR, train_set)
valid_ds = CustomSet(VIDEO_DIR, valid_set)
test_ds = CustomSet(VIDEO_DIR, test_set)

# definisco le batch_size
BATCH_TRAIN_SIZE, BATCH_TEST_SIZE = 64,64

# creo i dataloader
trainLoader = DataLoader(train_ds, batch_size=BATCH_TRAIN_SIZE, shuffle=True)
validLoader = DataLoader(valid_ds, batch_size=BATCH_TEST_SIZE, shuffle=True)
testLoader = DataLoader(test_ds, batch_size=BATCH_TEST_SIZE, shuffle=True)

# controllo se hanno la stessa forma di lista di tuple
print(len(trainLoader)*BATCH_TRAIN_SIZE, len(valid_set), len(test_set))
print(train_set[0])

23424 2596 11126
('0026091', 9)


Build modello
====

Rete prima
--- 

https://github.com/open-mmlab/mmaction/blob/master/MODEL_ZOO.md

In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer
path='/content/mmaction2/'
# Choose to use a config and initialize the recognizer
config = path+'configs/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb.py'
# Setup a checkpoint file to load

checkpoint = 'https://download.openmmlab.com/mmaction/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics_rgb/insvideo/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0',use_frames=False)

Use load_from_http loader


Downloading: "https://download.openmmlab.com/mmaction/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics_rgb/insvideo/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth" to /root/.cache/torch/hub/checkpoints/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth


In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
n_inputs = model.cls_head.fc_cls.in_features
model.cls_head.fc_cls = nn.Linear(n_inputs, 10)

Rete seconda
----

https://github.com/r1ch88/SlowFastNetworks/blob/master/lib/slowfastnet.py

In [ ]:
_all__ = ['resnet50', 'resnet101','resnet152', 'resnet200']



class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, head_conv=1):
        super(Bottleneck, self).__init__()
        if head_conv == 1:
            self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
            self.bn1 = nn.BatchNorm3d(planes)
        elif head_conv == 3:
            self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=(3, 1, 1), bias=False, padding=(1, 0, 0))
            self.bn1 = nn.BatchNorm3d(planes)
        else:
            raise ValueError("Unsupported head_conv!")
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=(1, 3, 3), stride=(1,stride,stride), padding=(0, 1, 1), bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out


class SlowFast(nn.Module):
    def __init__(self, block=Bottleneck, layers=[3, 4, 6, 3], class_num=10, dropout=0.5 ):
        super(SlowFast, self).__init__()

        self.fast_inplanes = 8
        self.fast_conv1 = nn.Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        self.fast_bn1 = nn.BatchNorm3d(8)
        self.fast_relu = nn.ReLU(inplace=True)
        self.fast_maxpool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.fast_res2 = self._make_layer_fast(block, 8, layers[0], head_conv=3)
        self.fast_res3 = self._make_layer_fast(
            block, 16, layers[1], stride=2, head_conv=3)
        self.fast_res4 = self._make_layer_fast(
            block, 32, layers[2], stride=2, head_conv=3)
        self.fast_res5 = self._make_layer_fast(
            block, 64, layers[3], stride=2, head_conv=3)
        
        self.lateral_p1 = nn.Conv3d(8, 8*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res2 = nn.Conv3d(32,32*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res3 = nn.Conv3d(64,64*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res4 = nn.Conv3d(128,128*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))

        self.slow_inplanes = 64+64//8*2
        self.slow_conv1 = nn.Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        self.slow_bn1 = nn.BatchNorm3d(64)
        self.slow_relu = nn.ReLU(inplace=True)
        self.slow_maxpool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.slow_res2 = self._make_layer_slow(block, 64, layers[0], head_conv=1)
        self.slow_res3 = self._make_layer_slow(
            block, 128, layers[1], stride=2, head_conv=1)
        self.slow_res4 = self._make_layer_slow(
            block, 256, layers[2], stride=2, head_conv=3)
        self.slow_res5 = self._make_layer_slow(
            block, 512, layers[3], stride=2, head_conv=3)
        self.dp = nn.Dropout(dropout)
        self.fc = nn.Linear(self.fast_inplanes+2048, class_num, bias=False)
    def forward(self, input ):
        fast, lateral = self.FastPath(input[:, :, ::2, :, :])
        slow = self.SlowPath(input[:, :, ::16, :, :], lateral)
        x = torch.cat([slow, fast], dim=1)
        x = self.dp(x)
        x = self.fc(x)
        return x



    def SlowPath(self, input, lateral):
        x = self.slow_conv1(input)
        x = self.slow_bn1(x)
        x = self.slow_relu(x)
        x = self.slow_maxpool(x)
        x = torch.cat([x, lateral[0]],dim=1)
        x = self.slow_res2(x)
        x = torch.cat([x, lateral[1]],dim=1)
        x = self.slow_res3(x)
        x = torch.cat([x, lateral[2]],dim=1)
        x = self.slow_res4(x)
        x = torch.cat([x, lateral[3]],dim=1)
        x = self.slow_res5(x)
        x = nn.AdaptiveAvgPool3d(1)(x)
        x = x.view(-1, x.size(1))
        return x

    def FastPath(self, input):
        lateral = []
        x = self.fast_conv1(input)
        x = self.fast_bn1(x)
        x = self.fast_relu(x)
        pool1 = self.fast_maxpool(x)
        lateral_p = self.lateral_p1(pool1)
        lateral.append(lateral_p)

        res2 = self.fast_res2(pool1)
        lateral_res2 = self.lateral_res2(res2)
        lateral.append(lateral_res2)
        
        res3 = self.fast_res3(res2)
        lateral_res3 = self.lateral_res3(res3)
        lateral.append(lateral_res3)

        res4 = self.fast_res4(res3)
        lateral_res4 = self.lateral_res4(res4)
        lateral.append(lateral_res4)

        res5 = self.fast_res5(res4)
        x = nn.AdaptiveAvgPool3d(1)(res5)
        x = x.view(-1, x.size(1))

        return x, lateral

    def _make_layer_fast(self, block, planes, blocks, stride=1, head_conv=1):
        downsample = None
        if stride != 1 or self.fast_inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(
                    self.fast_inplanes,
                    planes * block.expansion,
                    kernel_size=1,
                    stride=(1,stride,stride),
                    bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.fast_inplanes, planes, stride, downsample, head_conv=head_conv))
        self.fast_inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.fast_inplanes, planes, head_conv=head_conv))
        return nn.Sequential(*layers)

    def _make_layer_slow(self, block, planes, blocks, stride=1, head_conv=1):
        downsample = None
        if stride != 1 or self.slow_inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(
                    self.slow_inplanes,
                    planes * block.expansion,
                    kernel_size=1,
                    stride=(1,stride,stride),
                    bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.slow_inplanes, planes, stride, downsample, head_conv=head_conv))
        self.slow_inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.slow_inplanes, planes, head_conv=head_conv))
  
        self.slow_inplanes = planes * block.expansion + planes * block.expansion//8*2
        return nn.Sequential(*layers)




def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = SlowFast(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

model = SlowFast(class_num=400)

In [ ]:
model

In [ ]:
n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 10)

In [ ]:
model

Training modello
====

In [ ]:
def train_one_epoch(model, optimizer, trainloader, epoch, num_epochs):
  model.train()
  running_loss = 0.0
  train_correct = 0
  
  for batch_i,item in enumerate(trainloader):
    video, target = item['video'], item['label']
    target = torch.reshape(target, (-1,))
    video, target = video.cuda(), target.cuda()
    optimizer.zero_grad()
    outputs = model(video)  # senza SlowFast ResNet50 è outputs=model(video)

    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    _, predicted = torch.max(outputs.data, 1)
    correct_lbls = (predicted == target).sum().item()
    train_correct = train_correct + correct_lbls
    running_loss = running_loss + loss.item()

    # Print log
    sys.stdout.write(
      "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)] [lr: %.7f%%]"
      % (
        epoch+1,
        num_epochs,
        batch_i,
        len(trainloader),
        loss.item(),
        running_loss/(1+batch_i),
        correct_lbls/float(BATCH_TRAIN_SIZE),
        train_correct/float(BATCH_TRAIN_SIZE*(1+batch_i)),
        optimizer.param_groups[0]['lr']
      )
    )


  return [running_loss, train_correct]


def evaluate(model, validloader, isTestingPhase):
  model.eval()
  running_loss = 0.0
  valid_correct = 0
  if isTestingPhase:
    predicts, labels = [],[]
  
  with torch.no_grad():
    for item in validloader:
      video, target = item['video'], item['label']
      target = torch.reshape(target, (-1,))
      
      video, target = video.cuda(), target.cuda()
      # INFERENCE CALCULATING
      outputs = model(video) # senza SlowFast ResNet50 è outputs=model(video)
      val_loss = criterion(outputs, target)
      # MONITORING ACCURACY AND LOSS      
      _, predicted = torch.max(outputs.data, 1)
      valid_correct = valid_correct + (predicted == target).sum().item()
      running_loss += val_loss.item()

      if isTestingPhase:
        predicts.append(predicted.item())
        labels.append(target.item())

  if isTestingPhase:
    return [(predicts, labels), running_loss, valid_correct]
  
  else:
    return [None, running_loss, valid_correct]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # DOPO AVER ESEGUITO QUESTA CELLA RIAVVIARE IL RUNTIME!
!git clone "https://github.com/facebookresearch/VMZ"
!pip install -e VMZ/pt/
# # DOPO AVER ESEGUITO QUESTA CELLA RIAVVIARE IL RUNTIME!

In [ ]:
from vmz.models import ir_csn_152
model = ir_csn_152(pretraining="ig_ft_kinetics_32frms")

for param in model.parameters():
    param.requires_grad = False

n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 10)

In [ ]:
torch.cuda.empty_cache()

model.cuda()

lr = 1e-3
print("parametri training (",new_H,new_W,")","lr:",lr,"batch_size:",BATCH_TRAIN_SIZE)
# costruisco i pesi per la loss per il dataset sbilanciato
y_train = [v for k,v in train_set]
weights = torch.FloatTensor(compute_class_weight('balanced', np.unique(y_train), y_train )).cuda()

# costruisco optimizer, criterio di loss, parametri di training

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

EPOCHS=50
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # classico
# lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, min_lr=1e-6) # varia lr su plateau

tr_loss, val_loss = [], []
tr_accs, val_accs = [], []

for epoch in range(EPOCHS):
  # train for one epoch
  start_time = time.time()
  train_loss, train_correct = train_one_epoch(model, optimizer, trainLoader, epoch, EPOCHS)
  end_time = time.time() - start_time

  # validate on validation set
  _, valid_loss, valid_correct = evaluate(model, validLoader, isTestingPhase=False)

  # lr_scheduler.step( valid_loss/float(len(validLoader)) )
  lr_scheduler.step()
  print(
      f' [Val_loss = {valid_loss/float(len(validLoader)):0.7f},'
      f' Val_acc = {valid_correct/float(len(validLoader)*BATCH_TEST_SIZE):0.7f}]',
      f'in {end_time:.2f} sec'
  )

  tr_loss.append(train_loss/float(len(trainLoader)))
  val_loss.append(valid_loss/float(len(validLoader)))
  tr_accs.append(float(train_correct)/float(len(trainLoader)*BATCH_TRAIN_SIZE))
  val_accs.append(float(valid_correct)/float(len(validLoader)*BATCH_TEST_SIZE))

  checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            'scheduler':lr_scheduler.state_dict()
  }

  path="drive/MyDrive/"
  torch.save(checkpoint, path+'model_train_e'+str(epoch)+'.pth')

plt.plot(tr_loss, label='Training loss', c='r')
plt.plot(val_loss, label='Validation loss', c='b')
plt.legend(frameon=False)
plt.show()

plt.plot(tr_accs, label='Training accuracy', c='r')
plt.plot(val_accs, label='Validation accuracy', c='b')
plt.legend(frameon=False)
plt.show()

In [ ]:
output

Testing
====

In [ ]:
outputTargetTuple, test_loss, test_correct = evaluate(model, testLoader, isTestingPhase=True)
print(f'Accuracy = {(test_correct/float(len(testLoader)*BATCH_TEST_SIZE))*100}%')
print(classification_report(y_true=outputTargetTuple[0], y_pred=outputTargetTuple[1], zero_division=0))

In [ ]:
confusion_matrix( y_true=outputTargetTuple[0], y_pred=outputTargetTuple[1], labels=np.arange(0,10) )

In [ ]:
print(outputTargetTuple[0])
print(outputTargetTuple[1])